In [1]:
import random
import tensorflow as tf
import tensorflow.keras.backend as K

import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, Embedding, Concatenate, Reshape, Lambda
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import OrdinalEncoder

import dice_ml
from dice_ml.utils import helpers # helper functions

from alibi.explainers import CounterFactualProto
from alibi.utils.mapping import ohe_to_ord, ord_to_ohe

import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import time

from xmoai.setup.configure import generate_counterfactuals_regression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
seed = 2020
tf.keras.backend.clear_session()
np.random.seed(seed) # NumPy
random.seed(seed) # Python
tf.random.set_seed(seed) # Tensorflow

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<h5>Loading the dataset, models and metadata</h5>

In [4]:
target_col = 'isFraud'

[xgb, lgb, rf, X_val_encoded, y_val, cat_columns, cat_object_columns, cat_vars_ord] = pickle.load(open('training_data.pkl', 'rb'))
[X_test, y_test, correct_indexes] = pickle.load(open('validation_dataset.pkl', 'rb'))

#model = tf.keras.models.load_model('model_reg')

<h5>xMOAI - scikit-learn</h5>

In [5]:
def get_features_from_instance_for_xmoai(query_instance_xmoai, columns):
    # xMOAI: informing which features can be changed and their allowed ranges
    immutable_cols = []
    upper_bounds = query_instance_xmoai.copy()
    lower_bounds = query_instance_xmoai.copy()
    categorical_columns_xmoai = {}

    for col in cat_vars_ord.keys():
        count = cat_vars_ord[col]
        categorical_columns_xmoai[col-1] = list(range(count))
        
        lower_bounds[col-1] = 0
        upper_bounds[col-1] = count-1
        
        if col <= 3 or (col >= 9 and col <= 305):
            immutable_cols.append(col-1)
    
    return immutable_cols, upper_bounds, lower_bounds, categorical_columns_xmoai

In [6]:
xmoai_results_rf = {}

counter = 1
for index in correct_indexes:
    display(f'Individual {counter} of {len(correct_indexes)}')
    counter += 1
    
    current_cfs = []
    current_front = []
    current_algs = []
    recorded_times = np.array([])

    X_current_xmoai = X_test.loc[index:index]
    query_instance_xmoai = X_current_xmoai.values[0]

    # get prediction information
    current_prediction = rf.predict(np.array([query_instance_xmoai]))[0]

    y_desired = current_prediction-10
    y_acceptable_range = np.array([y_desired-10, current_prediction-1])
    immutable_cols, upper_bounds, lower_bounds, categorical_columns_xmoai = get_features_from_instance_for_xmoai(query_instance_xmoai, X_current_xmoai.columns)
    max_changed_vars = len(query_instance_xmoai) - len(immutable_cols)
    integer_columns = []
        
    run = 1
    limit = 30
    for i in list(range(max_changed_vars, 16, -2)):
        display(f'Run {run} of {limit} (maximum immutable columns: {i})')
    
        start_time = time.time()
        pareto_front, pareto_set, pareto_algorithms = generate_counterfactuals_regression(rf, query_instance_xmoai,
                                                            y_desired, immutable_cols, y_acceptable_range, 
                                                            upper_bounds, lower_bounds, categorical_columns_xmoai,
                                                            integer_columns, method_name='predict', max_changed_vars=i,
                                                            verbose=False, seed=seed, n_jobs=-1, pop_size=100, n_gen=50)
        
        try:
            current_cfs.append(pareto_set.tolist())
            current_front.append(pareto_front.tolist())
            current_algs.append(pareto_algorithms.tolist())
            display(f'Run finished.')
        except:
            display(f'Run finished with errors.')
        
        recorded_times = np.append(recorded_times, time.time() - start_time)

        run += 1

    xmoai_results_rf[index] = (current_cfs, current_front, current_algs, recorded_times)

'Individual 1 of 10'

'Run 1 of 30 (maximum immutable columns: 76)'


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.configuration import Configuration
Configuration.show_compile_hint = False



'Run finished.'

'Run 2 of 30 (maximum immutable columns: 74)'

...

'Run finished.'

'Run 29 of 30 (maximum immutable columns: 20)'

'Run finished.'

'Run 30 of 30 (maximum immutable columns: 18)'

'Run finished.'

In [7]:
pickle.dump(xmoai_results_rf, open('xmoai_results_rf.pkl', 'wb'))